**Import Libraries**

This cell imports all the necessary libraries and modules required to build a retrieval-augmented question-answering pipeline for neuroscience literature. The imports include modules for data retrieval from external sources (arXiv, bioRxiv), text preprocessing, document embedding, and vector storage. Additionally, libraries are included for working with large language models (transformers, quantization), constructing retrieval and generation chains, and deploying the interactive web interface using Gradio. These components together enable the system to fetch scientific abstracts, process and embed them for semantic search, and generate reliable, context-aware answers using advanced language models.

In [3]:
import re
import arxiv
import requests
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import pipeline
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr

**Define Neuroscience Topics**

This cell lists core topics in neuroscience and neuroengineering to guide literature searches. These keywords are used to retrieve relevant research abstracts from public databases, ensuring broad coverage of important areas in the field.

In [4]:
# Define neuroscience topics
NEURO_TOPICS = [
    "neuroscience", "neuroengineering", "brain-computer interface",
    "neuroprosthetics", "cognitive neuroscience", "motor control",
    "neuroimaging", "fMRI", "EEG", "neural decoding", "neural interfaces",
    "computational neuroscience", "synaptic plasticity", "learning and memory",
    "brain signal processing", "spiking neural networks", "neuromodulation"
]

**Clean Raw Text**

To ensure consistent input for the retrieval pipeline, this function preprocesses and standardizes raw text by removing extra whitespace and unwanted characters. By cleaning and formatting the research abstracts, it improves the quality of embeddings and enhances the accuracy of document retrieval and language model responses.

In [5]:
# Clean raw text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,;:!?()-]', '', text)
    return text.strip()


**Fetch Abstracts from arXiv**

This function automates the retrieval of research abstracts from the arXiv database based on a list of neuroscience-related topics. For each topic, it queries arXiv for recent papers, extracts the title and abstract, and applies text cleaning for consistency. The collected and processed abstracts are then returned as a list, forming the basis for downstream embedding and retrieval tasks.

In [6]:
# Fetch abstracts from arXiv
def fetch_arxiv_abstracts(topics, max_per_topic=2):
    abstracts = []
    for topic in topics:
        search = arxiv.Search(
            query=topic,
            max_results=max_per_topic,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )
        for result in search.results():
            raw = f"Title: {result.title}\nAbstract: {result.summary}"
            abstracts.append(clean_text(raw))
    return abstracts

**bioRxiv Abstract Collection**

To expand the literature base beyond arXiv, this function gathers recent neuroscience abstracts from the bioRxiv preprint server. It queries the bioRxiv API for each specified topic, extracts and cleans the title and abstract from each retrieved paper, and compiles the results into a list

In [7]:
# Fetch abstracts from bioRxiv
def fetch_biorxiv_abstracts(topics, max_per_topic=2):
    abstracts = []
    for topic in topics:
        url = f"https://api.biorxiv.org/details/biorxiv/{topic}/0/{max_per_topic}"
        try:
            res = requests.get(url).json()
            for paper in res.get("collection", []):
                raw = f"Title: {paper['title']}\nAbstract: {paper['abstract']}"
                abstracts.append(clean_text(raw))
        except Exception:
            continue
    return abstracts

**Fetch and Clean Abstracts**

Retrieval and cleaning of neuroscience research abstracts from arXiv and bioRxiv are performed using the defined functions for each target topic. After gathering and processing the papers, the results are merged into a single dataset with consistent formatting. The total number of cleaned abstracts is displayed to verify that data aggregation has been completed successfully.

In [8]:
# Fetch and clean abstracts
print("📘 Fetching from arXiv...")
arxiv_docs = fetch_arxiv_abstracts(NEURO_TOPICS, max_per_topic=100)

📘 Fetching from arXiv...


/tmp/ipython-input-266927979.py:10: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


In [9]:
print("🧪 Fetching from bioRxiv...")
biorxiv_docs = fetch_biorxiv_abstracts(NEURO_TOPICS, max_per_topic=100)

🧪 Fetching from bioRxiv...


In [10]:
all_cleaned_docs = arxiv_docs + biorxiv_docs
print(f"✅ Total cleaned abstracts: {len(all_cleaned_docs)}")

✅ Total cleaned abstracts: 1556


**Chunk the Cleaned Documents**

To prepare the research abstracts for semantic search and retrieval, this section splits each cleaned document into smaller, overlapping text chunks. By breaking long texts into manageable segments, it ensures that relevant information can be efficiently indexed and retrieved during question answering. The chunking parameters are set to balance context and granularity for optimal embedding performance.

In [11]:
# Chunk the cleaned documents
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)
chunks = []
for doc in all_cleaned_docs:
    chunks.extend(text_splitter.split_text(doc))

**Embed and Store Using Chroma**

By converting each text chunk into a vector representation with Hugging Face embeddings, the workflow enables efficient similarity search over the dataset. These embeddings are saved in a Chroma vector database, and a retriever is set up to rapidly find and return the most relevant document sections during question answering.

In [12]:
# Embed and store using Chroma
embedding = HuggingFaceEmbeddings()
vectorstore = Chroma.from_texts(chunks, embedding)
retriever = vectorstore.as_retriever()

/tmp/ipython-input-685671482.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
/tmp/ipython-input-685671482.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Load and Configure the Language Model**

This cell loads the Mistral-7B-Instruct-v0.2 large language model in 4-bit quantized mode to optimize memory efficiency on GPU hardware. The BitsAndBytesConfig enables efficient quantization, while the Hugging Face Transformers pipeline is used to build a text-generation interface around the model. After loading the tokenizer and configuring the generation parameters, a sample prompt is used to verify the model’s ability to answer neuroscience questions, demonstrating the LLM’s knowledge about the thalamus and its functions.

In [13]:
# Load Hugging Face model (Mistral-7B-Instruct-v0.2 in 4-bit quantization)
print("📥 Loading Hugging Face model: mistralai/Mistral-7B-Instruct-v0.2 in 4-bit quantization")
# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create the pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.0, # Keep temperature at 0 for greedy decoding
    do_sample=False,
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Test it
response = llm("What are the roles of the thalamus in the brain?")
print(response)

📥 Loading Hugging Face model: mistralai/Mistral-7B-Instruct-v0.2 in 4-bit quantization


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-2902563992.py:31: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)
/tmp/ipython-input-2902563992.py:34: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm("What are the roles of the thalamus in the brain?")
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=inf

What are the roles of the thalamus in the brain?

The thalamus is a large, complex structure located in the midbrain that plays a crucial role in the processing and relaying of sensory information to the cerebral cortex. It acts as a relay station for sensory information from various parts of the body, including the senses of touch, taste, sight, hearing, and smell. The thalamus also plays a role in motor control, memory, attention, and consciousness. It receives information from the sensory organs and sends it to the appropriate areas of the cerebral cortex for further processing. Additionally, the thalamus plays a role in regulating the sleep-wake cycle and in the coordination of motor movements. Damage to the thalamus can result in various neurological symptoms, including sensory deficits, motor impairments, and cognitive difficulties.


**Build RAG QA Chain**

To enable retrieval-augmented question answering, a RetrievalQA chain is constructed by combining the language model with the document retriever. This integration allows the system to answer queries using both the semantic capabilities of the LLM and relevant content extracted from the embedded neuroscience literature, ensuring responses are accurate and well-grounded in scientific sources.

In [14]:
# Build RAG QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

**Ask a Sample Question and Compare Answers**

A neuroscience-related question is submitted to both the retrieval-augmented generation (RAG) pipeline and the language model alone. The RAG chain answers the question using relevant content extracted from the most recent neuroscience literature, while the direct LLM response relies only on pre-trained knowledge. Comparing the two outputs shows that the RAG approach provides more current and contextually grounded information from recent papers, whereas the direct LLM response delivers accurate but less up-to-date insights based on general model knowledge.

In [15]:
# Ask a sample question
query = "What do recent neuroscience papers say about brain-computer interfaces?"
print("\n📌 Question:", query)

# RAG answer (with context)
rag_answer = qa.run(query)
print("\n🧠 Answer WITH RAG (using document retrieval):")
print(rag_answer)

# Direct LLM answer (no documents)
prompt = PromptTemplate.from_template("Answer the following question clearly:\n{question}")
llm_chain = LLMChain(llm=llm, prompt=prompt)
direct_answer = llm_chain.run(query)
print("\n🤖 Answer WITHOUT RAG (no document context):")
print(direct_answer)


📌 Question: What do recent neuroscience papers say about brain-computer interfaces?


/tmp/ipython-input-2428311421.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_answer = qa.run(query)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/tmp/ipython-input-2428311421.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🧠 Answer WITH RAG (using document retrieval):
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Title: On using AI for EEG-based BCI applications: problems, current challenges and future trends Abstract: Imagine unlocking the power of the mind to communicate, create, and even interact with the world around us. Recent breakthroughs in Artificial Intelligence (AI), especially in how machines see and understand language, are now fueling exciting progress in decoding brain signals from scalp electroencephalography (EEG). Prima facie, this opens the door to revolutionary brain-computer interfaces (BCIs) designed for real life, moving beyond traditional uses to envision Brain-to-Speech, Brain-to-Image, and even a Brain-to-Internet of Things (BCIoT). However, the journey is not as straightforward as it was for Computer Vision (CV) and Natural Language Processing (NLP). Applying A

**Deploy the Gradio Interface**

A Gradio function is defined to process user questions using the RAG pipeline and return contextually grounded answers. This function is then integrated into a user-friendly Gradio web interface, allowing users to interactively submit neuroscience queries and receive real-time responses based on the latest scientific literature from arXiv and bioRxiv. The resulting application provides an accessible and reliable tool for exploring neuroscience research through retrieval-augmented language model outputs.

In [16]:
# Gradio function
def answer(user_query):
    rag = qa.run(user_query)
    return rag

# Launch Gradio UI
gr.Interface(
    fn=answer,
    inputs=gr.Textbox(lines=3, placeholder="Ask a neuroscience question..."),
    outputs=[
        gr.Textbox(label="📚 Answer WITH RAG (arXiv + bioRxiv)"),
    ],
    title="🧠 Neuroscience Research Assistant",
    description="LLM enhanced with real scientific literature from arXiv and bioRxiv to deliver reliable answers in neuroscience."
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://528f1d859f4a7de367.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
